In [ ]:
import os
from pathlib import Path
import cv2
import yaml
import numpy as np
from box import Box
import matplotlib.pyplot as plt

# Config

## Path

In [ ]:
config_path = "ultralytics/cfg/datasets/aihub-val.yaml"
with open(config_path) as f:
    cfg = Box(yaml.safe_load(f))
    cfg["path"] = Path(cfg["path"])

In [ ]:
image_paths = cfg.path / cfg.images
label_paths = cfg.path / cfg.labels

In [ ]:
for dirpath, dirnames, filenames in os.walk(image_paths):
    for filename in sorted(filenames)[900:]:
        image_path = Path(dirpath) / filename
        _label_path = label_paths / filename
        label_path = _label_path.with_suffix(".txt")
        break

# File

## File

In [ ]:
image_path.exists(), label_path.exists()

In [ ]:
print(image_path), print(label_path)

## Load object

### Load image

In [ ]:
img_arr = cv2.imread(str(image_path))
ishape = img_arr.shape[1::-1]
fig, ax = plt.subplots()
ax.imshow(cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB))
ax.set_title(image_path.stem)
fig.show()

### Load label

In [ ]:
from ultralytics.utils.instance import Bboxes, Instances

In [ ]:
with open(label_path) as f:
    lines = f.readlines()
    lines = np.array([line.split(" ") for line in lines], dtype=float)  # ltwh
labels, _bboxes = lines[:, 0], lines[:, 1:]

# Preprocess

In [ ]:
# Bboxes
bboxes = Bboxes(_bboxes.copy(), format="xywh")
bboxes.convert("xyxy")
bboxes.mul(ishape * 2)

In [ ]:
# Instances
instances = Instances(_bboxes.copy(), bbox_format="xywh")
instances.convert_bbox("xyxy")
instances.denormalize(*ishape)

# Visualize

In [ ]:
from ultralytics.utils.plotting import Annotator

In [ ]:
ann = Annotator(img_arr)
for det in instances.bboxes:
    ann.box_label(det, label="Person")
plt.imshow(cv2.cvtColor(ann.im, cv2.COLOR_BGR2RGB))
plt.show()